<a href="https://colab.research.google.com/github/XinyueChen-Flora/NLP-Learn/blob/main/Genism_learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pprint

In [2]:
document = "Human machine interface for lab abc computer applications"

In [3]:
text_corpus = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

In [4]:
# Create a set of frequent words
stoplist = set('for a of the and to in'.split(' '))
# Lowercase each document, split it by white space and filter out stopwords
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in text_corpus]

# Count word frequencies
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

# Only keep words that appear more than once
processed_corpus = [[token for token in text if frequency[token] > 1] for text in texts]
pprint.pprint(processed_corpus)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


In [5]:
from gensim import corpora

dictionary = corpora.Dictionary(processed_corpus)
print(dictionary)

Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)


In [6]:
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]
pprint.pprint(bow_corpus)

[[(0, 1), (1, 1), (2, 1)],
 [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(2, 1), (5, 1), (7, 1), (8, 1)],
 [(1, 1), (5, 2), (8, 1)],
 [(3, 1), (6, 1), (7, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(4, 1), (10, 1), (11, 1)]]


In [7]:
from gensim import models

# train the model
tfidf = models.TfidfModel(bow_corpus)

# transform the "system minors" string
words = "system minors".lower().split()
print(tfidf[dictionary.doc2bow(words)])

[(5, 0.5898341626740045), (11, 0.8075244024440723)]


In [8]:
from gensim import similarities

index = similarities.SparseMatrixSimilarity(tfidf[bow_corpus], num_features=12)


In [9]:
query_document = 'system engineering'.split()
query_bow = dictionary.doc2bow(query_document)
sims = index[tfidf[query_bow]]
print(list(enumerate(sims)))

[(0, 0.0), (1, 0.32448703), (2, 0.41707572), (3, 0.7184812), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0)]


# Corpora and Vector Spaces

In [12]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

From Strings to Vectors

In [13]:
documents = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

In [14]:
from pprint import pprint  # pretty-printer
from collections import defaultdict

# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in documents
]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

pprint(texts)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


In [15]:
from gensim import corpora
dictionary = corpora.Dictionary(texts)
dictionary.save('/tmp/deerwester.dict')  # store the dictionary, for future reference
print(dictionary)

2022-01-02 20:59:55,702 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-01-02 20:59:55,705 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)
2022-01-02 20:59:55,709 : INFO : saving Dictionary object under /tmp/deerwester.dict, separately None
2022-01-02 20:59:55,713 : INFO : saved /tmp/deerwester.dict


Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)


In [16]:
print(dictionary.token2id)

{'computer': 0, 'human': 1, 'interface': 2, 'response': 3, 'survey': 4, 'system': 5, 'time': 6, 'user': 7, 'eps': 8, 'trees': 9, 'graph': 10, 'minors': 11}


In [17]:
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)  # the word "interaction" does not appear in the dictionary and is ignored


[(0, 1), (1, 1)]


In [18]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('/tmp/deerwester.mm', corpus)  # store to disk, for later use
print(corpus)

2022-01-02 21:01:39,329 : INFO : storing corpus in Matrix Market format to /tmp/deerwester.mm
2022-01-02 21:01:39,332 : INFO : saving sparse matrix to /tmp/deerwester.mm
2022-01-02 21:01:39,338 : INFO : PROGRESS: saving document #0
2022-01-02 21:01:39,340 : INFO : saved 9x12 matrix, density=25.926% (28/108)
2022-01-02 21:01:39,342 : INFO : saving MmCorpus index to /tmp/deerwester.mm.index


[[(0, 1), (1, 1), (2, 1)], [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(2, 1), (5, 1), (7, 1), (8, 1)], [(1, 1), (5, 2), (8, 1)], [(3, 1), (6, 1), (7, 1)], [(9, 1)], [(9, 1), (10, 1)], [(9, 1), (10, 1), (11, 1)], [(4, 1), (10, 1), (11, 1)]]


Creating a transformation

In [19]:
from gensim import models

tfidf = models.TfidfModel(corpus)  # step 1 -- initialize a model

2022-01-02 21:03:25,736 : INFO : collecting document frequencies
2022-01-02 21:03:25,740 : INFO : PROGRESS: processing document #0
2022-01-02 21:03:25,742 : INFO : calculating IDF weights for 9 documents and 11 features (28 matrix non-zeros)


In [20]:
doc_bow = [(0, 1), (1, 1)]
print(tfidf[doc_bow])  # step 2 -- use the model to transform vectors

[(0, 0.7071067811865476), (1, 0.7071067811865476)]


In [21]:
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print(doc)


[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(0, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.3244870206138555), (6, 0.44424552527467476), (7, 0.3244870206138555)]
[(2, 0.5710059809418182), (5, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)]
[(1, 0.49182558987264147), (5, 0.7184811607083769), (8, 0.49182558987264147)]
[(3, 0.6282580468670046), (6, 0.6282580468670046), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(4, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


In [23]:
lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)  # initialize an LSI transformation
corpus_lsi = lsi_model[corpus_tfidf]  # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi
lsi_model.print_topics(2)

2022-01-02 21:05:52,352 : INFO : using serial LSI version on this node
2022-01-02 21:05:52,355 : INFO : updating model with new documents
2022-01-02 21:05:52,358 : INFO : preparing a new chunk of documents
2022-01-02 21:05:52,361 : INFO : using 100 extra samples and 2 power iterations
2022-01-02 21:05:52,367 : INFO : 1st phase: constructing (12, 102) action matrix
2022-01-02 21:05:52,370 : INFO : orthonormalizing (12, 102) action matrix
2022-01-02 21:05:52,391 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2022-01-02 21:05:52,397 : INFO : computing the final decomposition
2022-01-02 21:05:52,399 : INFO : keeping 2 factors (discarding 47.565% of energy spectrum)
2022-01-02 21:05:52,407 : INFO : processed documents up to #9
2022-01-02 21:05:52,412 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2022-01-02 21:05:52,414 : INFO : topic #

[(0,
  '0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"'),
 (1,
  '-0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"response" + -0.320*"time" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"')]

In [24]:
# both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
for doc, as_text in zip(corpus_lsi, documents):
    print(doc, as_text)

[(0, 0.06600783396090236), (1, -0.5200703306361849)] Human machine interface for lab abc computer applications
[(0, 0.19667592859142352), (1, -0.7609563167700053)] A survey of user opinion of computer system response time
[(0, 0.08992639972446272), (1, -0.7241860626752509)] The EPS user interface management system
[(0, 0.07585847652178022), (1, -0.6320551586003424)] System and human system engineering testing of EPS
[(0, 0.10150299184980047), (1, -0.5737308483002959)] Relation of user perceived response time to error measurement
[(0, 0.7032108939378313), (1, 0.1611518021402562)] The generation of random binary unordered trees
[(0, 0.8774787673119832), (1, 0.16758906864659184)] The intersection graph of paths in trees
[(0, 0.9098624686818575), (1, 0.14086553628718762)] Graph minors IV Widths of trees and well quasi ordering
[(0, 0.6165825350569274), (1, -0.053929075663895584)] Graph minors A survey


In [25]:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)
doc = "Human computer interaction"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]  # convert the query to LSI space
print(vec_lsi)

2022-01-02 21:10:51,350 : INFO : using serial LSI version on this node
2022-01-02 21:10:51,364 : INFO : updating model with new documents
2022-01-02 21:10:51,371 : INFO : preparing a new chunk of documents
2022-01-02 21:10:51,381 : INFO : using 100 extra samples and 2 power iterations
2022-01-02 21:10:51,386 : INFO : 1st phase: constructing (12, 102) action matrix
2022-01-02 21:10:51,391 : INFO : orthonormalizing (12, 102) action matrix
2022-01-02 21:10:51,399 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2022-01-02 21:10:51,401 : INFO : computing the final decomposition
2022-01-02 21:10:51,403 : INFO : keeping 2 factors (discarding 43.156% of energy spectrum)
2022-01-02 21:10:51,404 : INFO : processed documents up to #9
2022-01-02 21:10:51,406 : INFO : topic #0(3.341): 0.644*"system" + 0.404*"user" + 0.301*"eps" + 0.265*"time" + 0.265*"response" + 0.240*"computer" + 0.221*"human" + 0.206*"survey" + 0.198*"interface" + 0.036*"graph"
2022-01-02 21:10:51,408 : INFO : topic #1(2

[(0, 0.46182100453271524), (1, -0.07002766527900053)]


In [26]:
from gensim import similarities
index = similarities.MatrixSimilarity(lsi[corpus])  # transform corpus to LSI space and index it

2022-01-02 21:11:25,232 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2022-01-02 21:11:25,236 : INFO : creating matrix with 9 documents and 2 features


In [27]:
index.save('/tmp/deerwester.index')
index = similarities.MatrixSimilarity.load('/tmp/deerwester.index')

2022-01-02 21:11:36,772 : INFO : saving MatrixSimilarity object under /tmp/deerwester.index, separately None
2022-01-02 21:11:36,779 : INFO : saved /tmp/deerwester.index
2022-01-02 21:11:36,783 : INFO : loading MatrixSimilarity object from /tmp/deerwester.index
2022-01-02 21:11:36,785 : INFO : loaded /tmp/deerwester.index


In [28]:
sims = index[vec_lsi]  # perform a similarity query against the corpus
print(list(enumerate(sims)))  # print (document_number, document_similarity) 2-tuples


[(0, 0.998093), (1, 0.93748635), (2, 0.9984453), (3, 0.98658866), (4, 0.90755945), (5, -0.12416792), (6, -0.1063926), (7, -0.09879464), (8, 0.05004177)]


In [29]:
sims = sorted(enumerate(sims), key=lambda item: -item[1])
for doc_position, doc_score in sims:
    print(doc_score, documents[doc_position])

0.9984453 The EPS user interface management system
0.998093 Human machine interface for lab abc computer applications
0.98658866 System and human system engineering testing of EPS
0.93748635 A survey of user opinion of computer system response time
0.90755945 Relation of user perceived response time to error measurement
0.05004177 Graph minors A survey
-0.09879464 Graph minors IV Widths of trees and well quasi ordering
-0.1063926 The intersection graph of paths in trees
-0.12416792 The generation of random binary unordered trees


# Word2vec

In [30]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

2022-01-02 21:16:45,737 : INFO : Creating /root/gensim-data


[==================================================] 100.0% 1662.8/1662.8MB downloaded


2022-01-02 21:22:54,452 : INFO : word2vec-google-news-300 downloaded
2022-01-02 21:22:54,458 : INFO : loading projection weights from /root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz
2022-01-02 21:29:51,671 : INFO : loaded (3000000, 300) matrix from /root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz


In [ ]:
for index, word in enumerate(wv.index_to_key):
    if index == 10:
        break
    print(f"word #{index}/{len(wv.index_to_key)} is {word}")